PEGASUS on BILLSUM

Pull in the Pegasus model from HuggingFace that has been finetuned on BillSum. Replicate their results on our test sample, and also show the difference between using our hyperparams for generation and their defaults. We can also see how the summary quality and readability clearly improves.

In [ ]:
# mount to colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q rouge_score
!pip install -q evaluate
import numpy as np
import evaluate
import rouge_score
import pandas as pd

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.4 MB/s eta 0:00:00


In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.9 MB/s eta 0:00:00


In [ ]:
import sentencepiece

In [ ]:
from transformers import AutoTokenizer  , TFAutoModel
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration
pmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-billsum", from_pt=True)
ptokenizer = PegasusTokenizer.from_pretrained("google/pegasus-billsum", from_pt=True)

from pprint import pprint

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# get regular test data
test_file = "drive/MyDrive/266project/billsum_test.csv"
test_data = pd.read_csv(test_file)

In [ ]:
#take a sample of 10 for testing my code, then a sample of 500 for the actual experiment
test_data_10 = test_data.sample(n=10, random_state=0)
test_data_500 = test_data.sample(n=500, random_state=0)
test_data_10 = test_data_10.reset_index(drop=True)
test_data_500 = test_data_500.reset_index(drop=True)

In [ ]:
# function for generation with better HPS
def generate_candidates_HPs(orig, model):
  """Test data is a pandas df, "orig" is test_df.orig.
  Adjusted hyperparameters are used.
  """
  input = ptokenizer(orig, return_tensors = 'tf',padding = True).input_ids
  output = model.generate(input,
                          max_new_tokens = 150,
                          min_length = 50,
                          num_beams = 1,
                          no_repeat_ngram_size = 2
                          )
  decoded_output = ptokenizer.decode(output[0], skip_special_tokens=True)
  return decoded_output

In [ ]:
test_data_10

,Unnamed: 0,orig,target,title
0,2730,SECTION 1. SHORT TITLE.\n\n This Act may be...,Forest Service Financial Accountability Restor...,Forest Service Financial Accountability Restor...
1,1515,SECTION 1. SHORT TITLE.\n\n This Act may be...,Desert Community Lands Act This bill directs t...,Desert Community Lands Act
2,2377,SECTION 1. SHORT TITLE.\n\n This Act may be...,Federal Land Freedom Act of 2015 This bill per...,Federal Land Freedom Act of 2015
3,654,SECTION 1. SHORT TITLE.\n\n This Act may be...,Mobile Workforce State Income Tax Fairness and...,To limit the authority of States to tax certai...
4,1998,SECTION 1. SHORT TITLE.\n This Act may be c...,Veterans' Compensation Cost-of-Living Adjustme...,"A bill to amend title 38, United States Code, ..."
5,882,SECTION 1. SHORT TITLE.\n\n This Act may be...,Foreign Credit Reform Act of 2004 - Amends Fed...,To provide multilateral and bilateral debt rel...
6,218,SECTION 1. SHORT TITLE.\n\n This Act may be...,Understanding the True Cost of College Act of ...,A bill to amend the Higher Education Opportuni...
7,1412,SECTION 1. SHORT TITLE.\n\n This Act may be...,Stop Legislative Automatic Spending Hikes (SLA...,To provide for the elimination of agencies and...
8,867,SECTION 1. SHORT TITLE.\n\n This Act may be...,Civil Justice Tax Fairness Act of 2013 - Amend...,Civil Justice Tax Fairness Act of 2013
9,2363,SECTION 1. SHORT TITLE.\n\n This Act may be...,Andrew Prior Act or Andrew's Law - Amends the ...,Andrew's Law


In [ ]:
test_data_10['peg_candidate'] = test_data_10.apply(lambda x: generate_candidates_HPs(x.orig, pmodel), axis = 1)

Token indices sequence length is longer than the specified maximum sequence length for this model (1166 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
test_data_10.to_csv('drive/MyDrive/266project/test_data_10_pegasus_cands.csv')

In [ ]:
peg_rouge_10 = rouge.compute(predictions = test_data_10['peg_candidate'],
                             references= test_data_10['target'])
peg_rouge_10


{'rouge1': 0.5921488880713328,
 'rouge2': 0.3772020700061098,
 'rougeL': 0.4636466767864881,
 'rougeLsum': 0.48671227868105893}

In [ ]:
#test_data_10.fillna("", inplace=True)

In [ ]:
test_data_10

In [ ]:
pd.set_option('display.max_colwidth', None)


In [ ]:
# use their default configs for generation and repeat

def generate_candidates(orig, model):
  """Test data is a pandas df, "orig" is test_df.orig.
  Default hyperparameters are used.
  """
  input = ptokenizer(orig, return_tensors = 'tf',padding = True).input_ids
  output = model.generate(input)
  decoded_output = ptokenizer.decode(output[0], skip_special_tokens=True)
  return decoded_output

In [ ]:
test_data_10['peg_candidate_2'] = test_data_10.apply(lambda x: generate_candidates(x.orig, pmodel), axis = 1)

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:854: UserWarning: Using `max_length`'s default (256) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [21]:
test_data_10.to_csv('drive/MyDrive/266project/test_data_10_pegasus_cands_2.csv')

In [22]:
peg_rouge_10_2 = rouge.compute(predictions = test_data_10['peg_candidate_2'],
                             references= test_data_10['target'])
peg_rouge_10_2

{'rouge1': 0.6069637964087351,
 'rouge2': 0.4160528612550477,
 'rougeL': 0.4936739250176195,
 'rougeLsum': 0.511803892982164}

In [ ]:
test_data_10.head(10)